In [1]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Mon Aug 22 09:00:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   46C    P0    67W / 300W |      2MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

## random search

## ADD dataset  
run_FFM_mask_ADD_ablation_hp.py

In [ ]:
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0 import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'add2022' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)

#dl.get_data(data_pick = '2', tde_pick = 't', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'd', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'e', pl_pick = 'l', to = 'd')


datapick = '1' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', to = 'e')

#track1 = data_loader(pathset)
#track1_generator = DataGenerator(track1.eval, track1.labels, **params_no_shuffle)

## hyperparameter search

## cut_hp : cutout, cutmix hp 탐색
cut_hps = [round(i,2) for i in np.arange(0.1,1,0.1)]


## spec_hp : specaug, specmix hp 탐색
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]
# #################################################
# ### get_ResMax
# mname = 'ResMax_ADD_'
# #################################################

# ### 1.Cutout
# for cut_hp in cut_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = cut_hp
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_ResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


# ### 2. CutMix  
# for cut_hp in cut_hps :       
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = cut_hp
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' :  specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_ResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


# ## 3. Specaug
# for spec_hp in spec_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = spec_hp
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_ResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
            
# ## 4. Specmix
# for spec_hp in spec_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = False
#         specmix = spec_hp
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_ResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)     
        
# ################################
# mname = "LCNN_ADD_"
# ################################

# ### 1.Cutout
# for cut_hp in cut_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = cut_hp
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_LCNN(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


# ### 2. CutMix  
# for cut_hp in cut_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = cut_hp
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' :  specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_LCNN(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


# ## 3. Specaug
# for spec_hp in spec_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = spec_hp
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_LCNN(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
    
# ## 4. Specmix
# for spec_hp in spec_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = False
#         specmix = spec_hp
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_LCNN(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)     
            



###############################################
mname = "BCResMax_ADD_"
###############################################
# ### 1.Cutout
# for cut_hp in cut_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = cut_hp
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_BCResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


# ### 2. CutMix  
# for cut_hp in cut_hps :       
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = cut_hp
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' :  specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_BCResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


## 3. Specaug
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = spec_hp
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
    
    
## 4. Specmix
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = spec_hp
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)     
            



###############################################
mname = "DDWSseq_ADD_"
###############################################

# ### 1.Cutout
# for cut_hp in cut_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = cut_hp
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_DDWSseq(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


# ### 2. CutMix  
# for cut_hp in cut_hps :      
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = cut_hp
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' :  specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_DDWSseq(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


# ## 3. Specaug
# for spec_hp in spec_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = spec_hp
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_DDWSseq(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
 ## 4. Specmix
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = spec_hp
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,   endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)   

cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1692/1692 [==============================] - 50s 25ms/step - loss: 0.8496 - auc: 0.9165
Epoch 2/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0945 - auc: 0.9977
Epoch 3/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0773 - auc: 0.9982
Epoch 4/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0321 - auc: 0.9993
Epoch 5/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0337 - auc: 0.9996
Epoch 6/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0367 - auc: 0.9996 0s - loss: 0.0368 -
Epoch 7/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0176 - auc: 0.9998
Epoch 8/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0166 - auc: 0.9996
Epoch 9/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0190 - auc: 0.9997
Epoch 10/70
1692/1692 [==============================] - 43s 25ms/step - loss: 0.0088 - a

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.0008755500213282455


...

EER: 0.847089609141613


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.71573385518591
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': False, 'cutmix': False, 'cutout': False, 'specaug': False, 'specmix': [[2, 5], [2, 5]], 'data_dir': '/home/ubuntu/data/ADD/', 'lowpass': False, 'highpass': False, 'ranfilter2': False, 'model': 'DDWSseq_ADD_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.8470', 'eer_eval': '0.7157', 'saved_model': 'saved_models/model_name!DDWSseq_ADD_!dropout_rate!na!human_weight!na!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!False!cutmix!False!cutout!False!specaug!False!specmix![[2, 5], [2, 5]]!lowpass!--!highpass!--!ranfilter2!--!0.8470.hdf5', 'tnow': '2022-08-25 08:49:58.340268'}
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1692/1692 [==============================] - 68s 37ms/step - loss: 1.3188 - auc: 0.6929
Epoch 2/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.9742 - auc: 0.8817
Epoch 3/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.9586 - auc: 0.8723
Epoch 4/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.9317 - auc: 0.8736
Epoch 5/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.8929 - auc: 0.8852
Epoch 6/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.8832 - auc: 0.8867
Epoch 7/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.8651 - auc: 0.8927
Epoch 8/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.8589 - auc: 0.8915
Epoch 9/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.8440 - auc: 0.8967
Epoch 10/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.8364 - auc: 0.8996
Epoch 11/

## filter aug

In [ ]:
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0 import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'add2022' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)

#dl.get_data(data_pick = '2', tde_pick = 't', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'd', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'e', pl_pick = 'l', to = 'd')


datapick = '1' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', to = 'e')



## hyperparameter search

## cut_hp : cutout, cutmix hp 탐색
cut_hps = [round(i,2) for i in np.arange(0.1,1,0.1)]

# filter augment band 
filt_hps = [[2,5],[3,6],[5,8]] 

## spec_hp : specaug, specmix hp 탐색
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]



#################################################
### get_ResMax
mname = "ResMax_ADD_"
#################################################

### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "step" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_ResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
            

### 5.filter aug -linear
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_ResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


################################
mname = "LCNN_ADD_"
################################

### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "step" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


###############################################
mname = "BCResMax_ADD_"
###############################################


### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "step" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)



###############################################
mname = "DDWSseq_ADD_"
###############################################

### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "step" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1692/1692 [==============================] - 48s 25ms/step - loss: 0.5814 - auc: 0.8858
Epoch 2/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0782 - auc: 0.9983
Epoch 3/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0436 - auc: 0.9988
Epoch 4/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0567 - auc: 0.9990
Epoch 5/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0281 - auc: 0.9996
Epoch 6/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0201 - auc: 0.9996
Epoch 7/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0263 - auc: 0.9998
Epoch 8/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0154 - auc: 0.9997
Epoch 9/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0192 - auc: 0.9994
Epoch 10/70
1692/1692 [==============================] - 43s 26ms/step - loss: 0.0116 - auc: 0.9997
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 1.922115865143903e-05


...

EER: 3.844231730287806e-05


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.232866927592955
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': False, 'cutmix': False, 'cutout': False, 'specaug': False, 'specmix': False, 'data_dir': '/home/ubuntu/data/ADD/', 'lowpass': False, 'highpass': False, 'filteraug': [[5, 8], 2, 'linear', [-1, 1]], 'ranfilter2': False, 'model': 'BCResMax_ADD_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '3.8442', 'eer_eval': '0.2328', 'saved_model': 'saved_models/model_name!BCResMax_ADD_!dropout_rate!na!human_weight!na!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!False!cutmix!False!cutout!False!specaug!False!specmix!False!lowpass!--!highpass!--!filteraug!--!ranfilter2!--!3.8442.hdf5', 'tnow': '2022-11-10 01:09:15.275406'}
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1692/1692 [==============================] - 74s 41ms/step - loss: 0.6451 - auc: 0.9420
Epoch 2/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.1297 - auc: 0.9958
Epoch 3/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.0553 - auc: 0.9990
Epoch 4/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.0441 - auc: 0.9993
Epoch 5/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.0351 - auc: 0.9994
Epoch 6/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.0333 - auc: 0.9994
Epoch 7/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.0210 - auc: 0.9997
Epoch 8/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.0175 - auc: 0.9998
Epoch 9/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.0199 - auc: 0.9994
Epoch 10/70
1692/1692 [==============================] - 69s 41ms/step - loss: 0.0230 - auc: 0.9992
Epoch 11/

In [1]:
!nvidia-smi

Mon Nov 14 16:18:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   34C    P0    38W / 300W |      2MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [2]:
## linear만 다시 돌려보기
# range 10으로 수정 


import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0 import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'add2022' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)

#dl.get_data(data_pick = '2', tde_pick = 't', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'd', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'e', pl_pick = 'l', to = 'd')


datapick = '1' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', to = 'e')



## hyperparameter search

## cut_hp : cutout, cutmix hp 탐색
cut_hps = [round(i,2) for i in np.arange(0.1,1,0.1)]

# filter augment band 
filt_hps = [[2,5],[3,6],[5,8]] 
# filt_hps = [[3,6],[5,8]] 

## spec_hp : specaug, specmix hp 탐색
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]



#################################################
### get_ResMax
mname = "ResMax_ADD_"
#################################################

# ### 5.filter aug -step
# for filt_hp in filt_hps :
#     for i in range(5) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
#         n_band2 = filt_hp # 저자는 [3,6]
#         min_bw2 = [2,4,6,8,10] # 저자는 6
#         f_type2 = "step" #np.random.choice(["step", "linear"])
#         db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
#         filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             'filteraug': filteraug2,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_ResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
            

### 5.filter aug -linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_ResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


################################
mname = "LCNN_ADD_"
################################

# ### 5.filter aug -step
# for filt_hp in filt_hps :
#     for i in range(5) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
#         n_band2 = filt_hp # 저자는 [3,6]
#         min_bw2 = [2,4,6,8,10] # 저자는 6
#         f_type2 = "step" #np.random.choice(["step", "linear"])
#         db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
#         filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             'filteraug': filteraug2,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_LCNN(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


###############################################
mname = "BCResMax_ADD_"
###############################################


# ### 5.filter aug -step
# for filt_hp in filt_hps :
#     for i in range(5) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
#         n_band2 = filt_hp # 저자는 [3,6]
#         min_bw2 = [2,4,6,8,10] # 저자는 6
#         f_type2 = "step" #np.random.choice(["step", "linear"])
#         db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
#         filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             'filteraug': filteraug2,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_BCResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)



###############################################
mname = "DDWSseq_ADD_"
###############################################

# ### 5.filter aug -step
# for filt_hp in filt_hps :
#     for i in range(5) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
#         n_band2 = filt_hp # 저자는 [3,6]
#         min_bw2 = [2,4,6,8,10] # 저자는 6
#         f_type2 = "step" #np.random.choice(["step", "linear"])
#         db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
#         filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': add2022,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             'filteraug': filteraug2,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': add2022
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_DDWSseq(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': add2022,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': add2022
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_ADD_abal/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1692/1692 [==============================] - 69s 38ms/step - loss: 0.2206 - auc: 0.9711
Epoch 2/70
1692/1692 [==============================] - 64s 38ms/step - loss: 0.0425 - auc: 0.9992
Epoch 3/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.0075 - auc: 0.9998
Epoch 4/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.0202 - auc: 0.9996
Epoch 5/70
1692/1692 [==============================] - 64s 38ms/step - loss: 0.0139 - auc: 0.9997
Epoch 6/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.0038 - auc: 0.9999
Epoch 7/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.0048 - auc: 0.9999
Epoch 8/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.0056 - auc: 0.9999
Epoch 9/70
1692/1692 [==============================] - 63s 37ms/step - loss: 0.0073 - auc: 0.9999
Epoch 10/70
1692/1692 [==============================] - 63s 37ms/step - loss: 8.3442e-04 - auc: 1.0000
Epoch

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.0027333537392551987


...

Epoch 1/70
1692/1692 [==============================] - 73s 40ms/step - loss: 1.1918 - auc: 0.7956
Epoch 2/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.6119 - auc: 0.9286
Epoch 3/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.4095 - auc: 0.9678
Epoch 4/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.2945 - auc: 0.9825
Epoch 5/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.2000 - auc: 0.9916
Epoch 6/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.1461 - auc: 0.9951
Epoch 7/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.0975 - auc: 0.9975
Epoch 8/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.1036 - auc: 0.9972
Epoch 9/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.0761 - auc: 0.9982
Epoch 10/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.0860 - auc: 0.9979
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.00031283747942971354


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.20945205479452053
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': False, 'cutmix': False, 'cutout': False, 'specaug': False, 'specmix': False, 'data_dir': '/home/ubuntu/data/ADD/', 'lowpass': False, 'highpass': False, 'filteraug': [[5, 8], 4, 'linear', [-6, 6]], 'ranfilter2': False, 'model': 'DDWSseq_ADD_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.0003', 'eer_eval': '0.2094', 'saved_model': 'saved_models/model_name!DDWSseq_ADD_!dropout_rate!na!human_weight!na!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!False!cutmix!False!cutout!False!specaug!False!specmix!False!lowpass!--!highpass!--!filteraug!--!ranfilter2!--!0.0003.hdf5', 'tnow': '2022-11-21 00:05:34.784964'}
